Signal 3: Price Breakout with volume confirmation

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from module import *

In [2]:
tickers = ["TSLA", "AAPL", "AMD"]
start = "2020-01-01"
end = "2024-01-01"

df_prices, df_changes = download_stock_price_data(tickers, start, end)
prices = df_prices["TSLA"]

[*********************100%***********************]  3 of 3 completed


In [3]:
def donchian_channel(prices, window_length=20):
    
    #Initilize
    prices = np.asarray(prices)
    highs = np.full_like(prices, np.nan, dtype=float)
    lows = np.full_like(prices, np.nan, dtype=float)
    
    for i in range(window_length - 1, len(prices)):
        highs[i] = np.max(prices[i - window_length + 1:i + 1])
        lows[i] = np.min(prices[i - window_length + 1:i + 1])
    
    return highs, lows

In [16]:
def donchian_signals(prices, window_length=20):
    
    signals = pd.DataFrame(index=prices.index)
    signals['signal'] = 0.0

    price_array = prices.to_numpy()
    highs, lows = donchian_channel(price_array, window_length)

    # Entry and exit conditions
    buy_signal = price_array > highs
    sell_signal = price_array < lows

    #Create position signal with holding logic
    position = np.zeros(len(price_array), dtype=float)
    holding = 0
    for i in range(len(prices)):
        if holding == 0 and buy_signal[i]:
            holding = 1
        elif holding == 1 and sell_signal[i]:
            holding = 0
        position[i] = holding

    signals['signal'] = position
    signals['position_change'] = signals['signal'].diff().fillna(0)
    signals.iloc[0, signals.columns.get_loc('position_change')] = 0

    return signals['signal'], signals

In [11]:
def compute_adx(prices, window):

    prices = np.asarray(prices)
    
    #Infer directional movement
    dm_pos = []
    dm_neg = []
    for i in range(1, len(prices)):
        price_diff = prices[i] - prices[i - 1]
        if price_diff > 0:
            dm_pos.append(price_diff)
            dm_neg.append(0)
        elif price_diff < 0:
            dm_pos.append(0)
            dm_neg.append(-price_diff)
        else:
            dm_pos.append(0)
            dm_neg.append(0)

    #Compute true ranges 
    true_ranges = []
    for i in range(1, len(prices)):
        high_low = prices[i] - prices[i - 1]
        high_close = abs(prices[i] - prices[i - 1])
        low_close = abs(prices[i] - prices[i - 1])
        true_ranges.append(max(high_low, high_close, low_close))

    #Wilder's smoothing to create directional index
    atr = [np.mean(true_ranges[:window])]
    di_pos = [np.mean(dm_pos[:window])]
    di_neg = [np.mean(dm_neg[:window])]   
    for i in range(window, len(dm_pos)):
        atr.append((atr[-1] * (window - 1) + true_ranges[i]) / window)
        di_pos.append((di_pos[-1] * (window - 1) + dm_pos[i]) / window)
        di_neg.append((di_neg[-1] * (window - 1) + dm_neg[i]) / window)
    di_pos = np.array(di_pos)
    di_neg = np.array(di_neg)   
    dx = np.abs((di_pos - di_neg) / (di_pos + di_neg)) * 100

    #Smooth DX to get ADX
    adx = np.full(len(prices), 20.0) #Initalize with neutral priyes for the 2* windowlength warmup phase

    #Smooth DX to get ADX
    adx[window*2-1] = np.mean(dx[:window])
    for i in range(window*2, len(dx)):
        adx[i] = (adx[i - 1] * (window - 1) + dx[i]) / window
        
    return adx.flatten()

In [22]:
def signal_03(prices, adx_window_length, donchian_window_length):
    
    signals = pd.DataFrame(index=prices.index)
    signals['signal'] = 0.0

    adx = compute_adx(prices, adx_window_length)
    donchian_sig, _ = donchian_signals(prices, donchian_window_length)
    donchian_sig = np.asarray(donchian_sig)

    #Custom tradig logic since adx only detects trends but not in which direction -> combine_two_subsignals() function doesn't work
    position = np.zeros(len(prices), dtype=float)
    holding = 0
    for i in range(len(prices)):
        if np.isnan(adx[i]):
            continue
        if holding == 0 and (donchian_sig[i] == 1 or adx[i] > 25):
            holding = 1
        elif holding == 1 and donchian_sig[i] == 0 and adx[i] > 25:
            holding = 0
        position[i] = holding

    signals['signal'] = position
    signals['position_change'] = signals['signal'].diff().fillna(0)
    signals.iloc[0, signals.columns.get_loc('position_change')] = 0

    return signals

In [ ]:
signals = signal_03(prices, 14, 20)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(signals)

In [37]:
signals_df = pd.DataFrame({
    1: signals2,
    2: signals["position_change"].to_numpy()
}, index=signals.index)

In [6]:
#Do gridsearch for best params
param_grid = {
    'adx_window_length': np.arange(5, 15),
    'donchian_window_length': np.arange(15, 30)}


for ticker in tickers:
    prices = df_prices[ticker]
    best_params, best_score, best_metrics, results = gridsearch_strategy(price=prices, param_grid=param_grid, signal_fn=signal_03, metric='cumret')
    print(ticker)
    print(best_metrics)

TSLA
(7.923283969394047, 1.2076778625617541, 0.7169198830361274, 0.03829580478658762)
AAPL
(3.033462933635219, 1.5545718974279095, 0.24665618356846486, 0.015382922012482708)
AMD
(9.396755944096588, 1.9081666576819083, 0.5881196730771234, 0.021239855441186575)
